# Калибровка

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import json

## Константы

Поменяем названия камер местами, для более удобного расчета, то есть теперь:  
_cam1_ - левая камера  
_cam2_ - правая камера

In [ ]:
S_MARKERS_DIST = 33e-3 / 6.0  # real world distance between markers in m
M_MARKERS_DIST = 67e-3 / 6.0  # real world distance between markers in m
GRID_SHAPE = (7, 7)
camera_params = {
    'focal_length_m': 80e-3,
    'img_resolution': (5120, 4096),
    'pixel_sz_x': 4.5e-6,  # in m
    'pixel_sz_y': 4.5e-6  # in m
}
stereo_params = {
    'baseline': 412e-3,  # in m
    'center_dist_to_obj': 639e-3,  # in m
    'cam1_dist_to_obj': 660e-3,  # in m
    'cam2_dist_to_obj': 641e-3  # in m
}

## Подготовка данных

Вручную отметил точку на маркерных досках, т.к. иначе получить координаты всех точек на маркерной доске, чтобы они формировали матрицу не удавалось (пробовал это сделать с помощью выделения Blob-ов, binary threshold, Hough Circle Transform, использовать модели на нейронных сетях - U-Net). Выделить маркеры в целом получалось, однако затем всё равно требуется вручную сопоставить их с координатами в системе координат относительно маркерной доски.

Итак, координаты маркерных точек находятся в файле *markers_coords.json*. На каждом изображении 49 точек, формирующих матрицу 7 на 7. Точки затем сопоставляются с координатами самой доски в реальном 3D пространстве *real_markers_coords*.

__ВАЖНО__  
'cam1_imgs_paths' - должен соответствовать пути к папке, в которой лежат изображения с левой камеры  
'cam2_imgs_paths' - соответственно, с правой

In [ ]:
imgs_folder = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                              'test_imgs')
cam1_imgs_paths = glob.glob(f"{imgs_folder}/cam2_*.jpg")
cam2_imgs_paths = glob.glob(f"{imgs_folder}/cam1_*.jpg")

In [ ]:
cam1_imgs = {}
cam2_imgs = {}

for img_name in sorted(cam1_imgs_paths):
    img = cv.imread(img_name, 0)
    img_name = os.path.splitext(os.path.basename(img_name))[0]
    cam1_imgs[img_name] = img

for img_name in sorted(cam2_imgs_paths):
    img = cv.imread(img_name, 0)
    img_name = os.path.splitext(os.path.basename(img_name))[0]
    cam2_imgs[img_name] = img

In [ ]:
plt.figure(figsize=(30, 20)) 
for i, img in enumerate(cam1_imgs.values()):
    plt.subplot(1, len(cam1_imgs), i + 1) 
    plt.imshow(cv.cvtColor(img, cv.COLOR_GRAY2RGB))
    plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(30, 20)) 
for i, img in enumerate(cam2_imgs.values()):
    plt.subplot(1, len(cam2_imgs), i + 1) 
    plt.imshow(cv.cvtColor(img, cv.COLOR_GRAY2RGB))
    plt.axis('off')
plt.show()

Создадим словарь, в котором будет определено расстояние между маркерами для каждого изображения

In [ ]:
img_name_to_markers_dist = {}

for img_name in cam1_imgs.keys():
    if img_name == 'cam2_4':
        img_name_to_markers_dist[img_name] = S_MARKERS_DIST
    else:
        img_name_to_markers_dist[img_name] = M_MARKERS_DIST

for img_name in cam2_imgs.keys():
    if img_name == 'cam1_4':
        img_name_to_markers_dist[img_name] = S_MARKERS_DIST
    else:
        img_name_to_markers_dist[img_name] = M_MARKERS_DIST

Загрузка координат маркерных точек для калибровочных изображений

In [ ]:
# Load the markers points for these images
with open('data/markers_coords.json', 'r') as file:
    markers_coords = json.load(file)

# Convert the string coordinates ("x, y") into tuples of integers
for img_name, coords in markers_coords.items():
    markers_coords[img_name] = [tuple(map(int, point.split(','))) for point in coords]

In [ ]:
def plot_img_w_markers(img, markers):
    plt.figure(figsize=(10, 10))
    plt.imshow(cv.cvtColor(img, cv.COLOR_GRAY2RGB))
    for marker_coords in np.array(markers, dtype=np.float32):
        plt.scatter(marker_coords[0], marker_coords[1], color='red', s=10)
    plt.show()

### Пример размеченной фотки

In [ ]:
img_name = 'cam2_5'
img = cam1_imgs[img_name]
plot_img_w_markers(img, markers_coords[img_name])

## Поиск внутренних параметров камер

### Вычисление camera matrix

Camera matrix можно вычислить исходя из следующих параметров:
1. фокусное расстояние: 80mm
2. размер пикселя: 4.5 µm × 4.5 µm
3. разрешение изображения: 5120 × 4096 

In [ ]:
def calculate_CM(focal_length_m, img_resolution, pixel_sz_x, pixel_sz_y):
    width, height = img_resolution

    f_x = focal_length_m / pixel_sz_x
    f_y = focal_length_m / pixel_sz_y

    c_x = width / 2.0
    c_y = height / 2.0

    CM = np.array([
        [f_x, 0, c_x],
        [0, f_y, c_y],
        [0, 0, 1]
    ])

    return CM

In [ ]:
CM = calculate_CM(**camera_params)
CM

### Поиск вектора искажений

У нас уже есть матрица камеры и вычислим коэффициенты искажения с использованием калибровочных изображений и отмеченных вручную маркерах на них (*test_imgs/* и *markers_coords.json*).

calibrateCamera принимает начальное предположение для матрицы камеры (CM_initial_guess) и вычисляет только вектор искажения при использовании определенных флагов.

In [ ]:
def calculate_distortion(imgs, img_resolution, 
                         markers_coords, markers_dist, grid_shape,
                         CM, dist_initial_guess=None):
    rows, columns = grid_shape

    real_markers = []  # 3D markers coords in real world for each image
    img_markers = []  # 2D markers coords in image plane for each image
    
    for img_name in imgs:
        # Get the 2D image marker coordinates
        markers = np.array(markers_coords[img_name], dtype=np.float32)
        img_markers.append(markers)

        # Create 3D real-world coordinates grid based on the specific markers_dist for this image
        real_markers_grid_coords = np.zeros((rows * columns, 3), np.float32)
        real_markers_grid_coords[:, :2] = np.mgrid[0:rows, 0:columns].T.reshape(-1, 2)
        real_markers_grid_coords *= markers_dist[img_name]  # Apply specific markers_dist for this image
    
        real_markers.append(real_markers_grid_coords)

    # Calibration flags: use initial guesses, fix focal length and principal point
    flags = cv.CALIB_USE_INTRINSIC_GUESS + cv.CALIB_FIX_FOCAL_LENGTH + cv.CALIB_FIX_PRINCIPAL_POINT
    ret, _, dist, _, _ = cv.calibrateCamera(real_markers, img_markers, img_resolution, 
                                            CM, dist_initial_guess, 
                                            flags=flags)

    print(f'RMSE: {ret}')
    return dist

In [ ]:
all_cam_imgs = {**cam1_imgs, **cam2_imgs}

In [ ]:
dist = calculate_distortion(
    all_cam_imgs, camera_params['img_resolution'], 
    markers_coords, img_name_to_markers_dist, GRID_SHAPE, 
    CM, np.array([0.01, 0, 0, 0, 0])
)
dist

Вектор искажений не похаж на правду, слишком экстремальные значения, будем исопльзовать кастомный вектор, оцененный из параметров объектива

In [ ]:
dist = np.array([0, 0, 0, 0, 0], dtype=np.float32)

### Поиск всех внутренних параметров с помощью OpenCV calibrateCamera

In [ ]:
def calculate_intrinsic(imgs, img_resolution, 
                         markers_coords, markers_dist, grid_shape,
                         CM_initial_guess, dist_initial_guess):
    rows, columns = grid_shape

    real_markers = []  # 3D markers coords in real world for each image
    img_markers = []  # 2D markers coords in image plane for each image
    
    for img_name in imgs:
        # Get the 2D image marker coordinates
        markers = np.array(markers_coords[img_name], dtype=np.float32)
        img_markers.append(markers)

        # Create 3D real-world coordinates grid based on the specific markers_dist for this image
        real_markers_grid_coords = np.zeros((rows * columns, 3), np.float32)
        real_markers_grid_coords[:, :2] = np.mgrid[0:rows, 0:columns].T.reshape(-1, 2)
        real_markers_grid_coords *= markers_dist[img_name]  # Apply specific markers_dist for this image
    
        real_markers.append(real_markers_grid_coords)

    # Calibration flags: use initial guesses, fix focal length and principal point
    flags = cv.CALIB_USE_INTRINSIC_GUESS
    ret, CM, dist, _, _ = cv.calibrateCamera(real_markers, img_markers, img_resolution, 
                                            CM_initial_guess, dist_initial_guess, 
                                            flags=flags)

    print(f'RMSE: {ret}')
    return CM, dist

In [ ]:
CM, dist = calculate_intrinsic(
    all_cam_imgs, camera_params['img_resolution'], 
    markers_coords, img_name_to_markers_dist, GRID_SHAPE, 
    CM, dist)

CM, dist

## Стерео-калибровка

### Manual estimation of R, T matrices given the organizator info

In [ ]:
def create_rotation_matrix(rx_deg, ry_deg, rz_deg):
    rx_rad = np.deg2rad(rx_deg)
    ry_rad = np.deg2rad(ry_deg)
    rz_rad = np.deg2rad(rz_deg)
    
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(rx_rad), -np.sin(rx_rad)],
        [0, np.sin(rx_rad), np.cos(rx_rad)]
    ])
    R_y = np.array([
        [np.cos(ry_rad), 0, np.sin(ry_rad)],
        [0, 1, 0],
        [-np.sin(ry_rad), 0, np.cos(ry_rad)]
    ])
    R_z = np.array([
        [np.cos(rz_rad), -np.sin(rz_rad), 0],
        [np.sin(rz_rad), np.cos(rz_rad), 0],
        [0, 0, 1]
    ])
    
    R = np.matmul(np.matmul(R_z, R_y), R_x)
    
    return R

In [ ]:
# Calculating R
# Set global coordinate system in between cameras
R1 = create_rotation_matrix(0, 18.5, 0)
R2 = create_rotation_matrix(0, -18.5, 0)

R = np.dot(R2, R1.T)

# Calculating T
C1 = np.array([-0.206, 0.0, 0.0]) # Center of cam1 coord system
C2 = np.array([0.206, 0.0, 0.0])  # Center of cam2 coord system

# Translation vector
t12_world = C2 - C1

T = np.dot(R1.T, t12_world)

R, T

### Calibration using OpenCV

In [ ]:
def stereo_calibrate(cam1_imgs, cam2_imgs, img_resolution, 
                     markers_coords, markers_dist, grid_shape, 
                     CM, dist,
                     R_guess=None, T_guess=None):
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 1000, 1e-6)
    
    rows, columns = grid_shape

    real_markers = []  # 3D points in real world space
    cam1_img_markers = []
    cam2_img_markers = []

    for cam1_img_name, cam2_img_name in zip(cam1_imgs, cam2_imgs):
        cam1_markers = markers_dist[cam1_img_name]

        # 3D world coordinates for each grid point based on markers distance for each image
        real_markers_grid_coords = np.zeros((rows * columns, 3), np.float32)
        real_markers_grid_coords[:, :2] = np.mgrid[0:rows, 0:columns].T.reshape(-1, 2)
        real_markers_grid_coords *= cam1_markers

        real_markers.append(real_markers_grid_coords)

        cam1_img_markers.append(np.array(markers_coords[cam1_img_name], dtype=np.float32))
        cam2_img_markers.append(np.array(markers_coords[cam2_img_name], dtype=np.float32))

    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC + cv.CALIB_USE_EXTRINSIC_GUESS
    
    ret, _, _, _, _, R, T, E, F, _, _, _ = cv.stereoCalibrateExtended(
        real_markers, 
        cam1_img_markers, 
        cam2_img_markers, 
        CM, dist,
        CM, dist,
        img_resolution,
        R_guess, T_guess,
        criteria=criteria, 
        flags=stereocalibration_flags
    )
 
    print(f"Reprojection Error (RMSE): {ret}")
    return R, T, E, F


R: The relative rotation matrix between the first and second cameras. This matrix describes how the second camera is rotated relative to the first camera.

T: The translation vector between the first and second cameras. This vector describes how far and in which direction the second camera is from the first camera.

E: The essential matrix. This matrix encodes the rotation and translation between the two cameras, and it can be used to compute the epipolar geometry (constraints for stereo matching).

F: The fundamental matrix. This is another way of representing the geometric relationship between the two cameras. It defines the epipolar lines in stereo matching.

In [ ]:
R, T, E, F = stereo_calibrate(cam1_imgs, cam2_imgs, camera_params['img_resolution'],
                              markers_coords, img_name_to_markers_dist, GRID_SHAPE,
                              CM, dist, R, T)

In [ ]:
R, T

Проверим baseline и углы поворота для полученных матриц

In [ ]:
def rotation_matrix_to_euler_angles(R):
    sy = np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)
    
    singular = sy < 1e-6
    
    if not singular:
        yaw = np.arctan2(R[2, 1], R[2, 2])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = np.arctan2(R[1, 0], R[0, 0])
    else:
        yaw = np.arctan2(-R[1, 2], R[1, 1])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = 0

    return np.degrees(np.array([yaw, pitch, roll]))

def show_baseline_and_rotation(T_calculated, R_calculated, real_baseline, R_real=None):
    calculated_baseline = np.linalg.norm(T)
    
    print(f"Calculated baseline: {calculated_baseline:.2f} m")
    print(f"Real baseline: {real_baseline:.2f} m")
    
    baseline_difference = np.abs(calculated_baseline - real_baseline)
    print(f"Baseline difference: {baseline_difference:.2f} m")
    
    euler_angles_calculated = rotation_matrix_to_euler_angles(R_calculated)
    print("\nCalculated Euler angles (Yaw, Pitch, Roll):")
    print(f"Yaw: {euler_angles_calculated[0]:.2f} degrees")
    print(f"Pitch: {euler_angles_calculated[1]:.2f} degrees")
    print(f"Roll: {euler_angles_calculated[2]:.2f} degrees")

In [ ]:
show_baseline_and_rotation(T, R, stereo_params['baseline'])

## Export the camera's intrinsic and extrinsic parameters

In [ ]:
calibration_data_json = {
    'CM': CM.tolist(),
    'dist': dist.tolist(),
    'R': R.tolist(),
    'T': T.tolist(),
    'E': E.tolist(),
    'F': F.tolist()
}

with open('data/calibration_data_real.json', 'w') as f:
    json.dump(calibration_data_json, f, indent=4)